In [7]:
6**6

46656

In [8]:
import stk
import pymongo
import stko
import os
import pickle
os.chdir('c:\\Users\\ma11115\\OneDrive - Imperial College London\\github_folder\\BO_polymers\\')

In [9]:
fragement_names = pickle.load(open('inputs_and_outputs.pkl', 'rb'))['fragement names']
fragement_names.values[0]

array(['PNLCAIFEPYYUCC-UHFFFAOYSA-N', 'PNLCAIFEPYYUCC-UHFFFAOYSA-N',
       'PNLCAIFEPYYUCC-UHFFFAOYSA-N', 'PNLCAIFEPYYUCC-UHFFFAOYSA-N',
       'PNLCAIFEPYYUCC-UHFFFAOYSA-N', 'PNLCAIFEPYYUCC-UHFFFAOYSA-N'],
      dtype=object)

In [193]:

# Connect to a MongoDB. This example connects to a local
# MongoDB, but you can connect to a remote DB too with
# MongoClient() - read the documentation for pymongo to see how
# to do that.
xtb_path=os.path.join('"C:\\Users\\ma11115\\OneDrive - Imperial College London\\xtb-6.5.0\\bin\\"',"xtb")
output_dir = os.path.join(os.getcwd(),'xtb_opt_output_dir')
client = pymongo.MongoClient( "mongodb://129.31.66.201/")
db = stk.MoleculeMongoDb(client, database='stk_mohammed_new',)
import stk
import socket
client = pymongo.MongoClient( "mongodb://129.31.66.201/")
import re
def get_property_value(data, property_name):
    for line in data:
        if property_name in line:
            if property_name == 'cpu-time':
                property_value = re.findall(
                    r"[-+]?(?:\d*\.*\d+)", line)[-3] + " h "+re.findall(
                    r"[-+]?(?:\d*\.*\d+)", line)[-2] + " min " + re.findall(
                    r"[-+]?(?:\d*\.*\d+)", line)[-1]+ " s "
                return property_value
            property_value = float(re.findall(
                r"[-+]?(?:\d*\.*\d+)", line)[-1])  # float(words[3]) #
            return property_value
def save_xtb_opt_calculation(polymer, xtb_opt_output_dir,database='stk_mohammed_BO',collection='test'):
        
    polymer_xtb_opt_calc={'InChIKey':stk.get_inchi_key(polymer),
                            'xtb_opt_folder':os.path.join(xtb_opt_output_dir,stk.get_inchi_key(polymer)),
                            'Host IP':'Mohammeds_computer',  
                            }
    outfile = open(os.path.join(polymer_xtb_opt_calc['xtb_opt_folder'],'optimization_1.output'), 'r', encoding="utf8")
    data = outfile.readlines()
    outfile.close()

    polymer_xtb_opt_calc['cpu time']=get_property_value(data, 'cpu-time')
    polymer_xtb_opt_calc['total energy (au)']=get_property_value(data, 'TOTAL ENERGY')
    polymer_xtb_opt_calc['HOMO-LUMO GAP (eV)']=get_property_value(data, 'HOMO-LUMO GAP')


    collection=client[database][collection]

    collection.update_many(filter={'InChIKey':stk.get_inchi_key(polymer)},update={'$set':polymer_xtb_opt_calc},upsert=True)
def Build_polymer(polymer_id,num_fragments=6):
    precursors = []
    genes="ABCDEFGH"
    genes=genes[:num_fragments]
    repeating_unit = ""
    # joins the Genes to make a repeating unit string
    repeating_unit = repeating_unit.join(genes)
    print(repeating_unit)
    for fragment in fragement_names.values[polymer_id]:
        mol = db.get({'InChIKey':fragment})
        bb = stk.BuildingBlock.init_from_molecule(mol, functional_groups=[stk.BromoFactory()])
        precursors.append(bb)
    polymer = stk.ConstructedMolecule(
        stk.polymer.Linear(
            building_blocks=precursors,
            repeating_unit=repeating_unit,
            num_repeating_units=1,
            optimizer=stk.MCHammer()
        )
    )
    return polymer
def run_xtb_opt(polymer,xtb_path,xtb_opt_output_dir
                ,database='stk_mohammed_BO',collection='test'):
    collection=client[database][collection]
    if collection.find_one({'InChIKey':stk.get_inchi_key(polymer)}) is not None:
        print('already calculated')
        return polymer
    xtb = stko.OptimizerSequence(
        stko.MMFF(),
        stko.ETKDG(),
        stko.XTB(xtb_path=xtb_path,
               output_dir = os.path.join(xtb_opt_output_dir,stk.get_inchi_key(polymer)),
                unlimited_memory=False)
    )
    polymer = xtb.optimize(polymer)
    polymer.to_rdkit_mol()
    save_xtb_opt_calculation(polymer, xtb_opt_output_dir,database='stk_mohammed_BO',collection='test')

    return polymer
def run_xtb_ipea(polymer,xtb_path,xtb_opt_output_dir):
    
    xtb = XTBEnergy2(xtb_path=xtb_path,
            output_dir = xtb_opt_output_dir,
        unlimited_memory=False,
        calculate_ip_and_ea=True
    )
    xtb_results = xtb.get_results_2(polymer)
    XTB_results={
        'total energy (au)':xtb_results.get_total_energy(),
        'homo lumo_gap (eV)':xtb_results.get_homo_lumo_gap(),
        'electron affinity (eV)' : xtb_results.get_electron_affinity(),
        'ionisation potential (eV)':xtb_results.get_ionisation_potential(),
        'InChIKey':stk.get_inchi_key(polymer),
        'xtb_cal_folder':os.path.join(xtb_opt_output_dir,stk.get_inchi_key(polymer)),
        'Host IP':'Mohammeds_computer'}
    return XTB_results

xtb_opt_output_dir = 'xtb_opt_output_dir'
polymer =  Build_polymer(polymer_id = 1,num_fragments=6)
run_xtb_opt(polymer,xtb_path,xtb_opt_output_dir
                ,database='stk_mohammed_BO',collection='test')

ABCDEF


In [190]:
import stk
import socket
client = pymongo.MongoClient( "mongodb://129.31.66.201/")
import re
def get_property_value(data, property_name):
    for line in data:
        if property_name in line:
            if property_name == 'cpu-time':
                property_value = re.findall(
                    r"[-+]?(?:\d*\.*\d+)", line)[-3] + " h "+re.findall(
                    r"[-+]?(?:\d*\.*\d+)", line)[-2] + " min " + re.findall(
                    r"[-+]?(?:\d*\.*\d+)", line)[-1]+ " s "
                return property_value
            property_value = float(re.findall(
                r"[-+]?(?:\d*\.*\d+)", line)[-1])  # float(words[3]) #
            return property_value
def save_xtb_opt_calculation(polymer, xtb_opt_output_dir,database='stk_mohammed_BO',collection='test'):
        
    polymer_xtb_opt_calc={'InChIKey':stk.get_inchi_key(polymer),
                            'xtb_opt_folder':xtb_opt_output_dir+'/'+stk.get_inchi_key(polymer),
                            'Host IP':'Mohammeds_computer',  
                            }
    outfile = open(polymer_xtb_opt_calc['xtb_opt_folder']+'\\optimization_1.output', 'r', encoding="utf8")
    data = outfile.readlines()
    outfile.close()

    polymer_xtb_opt_calc['cpu time']=get_property_value(data, 'cpu-time')
    polymer_xtb_opt_calc['total energy (au)']=get_property_value(data, 'TOTAL ENERGY')
    polymer_xtb_opt_calc['HOMO-LUMO GAP (eV)']=get_property_value(data, 'HOMO-LUMO GAP')


    collection=client[database][collection]

    collection.update_many(filter={'InChIKey':stk.get_inchi_key(polymer)},update={'$set':polymer_xtb_opt_calc},upsert=True)

save_xtb_opt_calculation(polymer, xtb_opt_output_dir,database='stk_mohammed_BO',collection='test')


In [111]:
# Calculate energy using GFN-xTB.

output_dir = os.path.join(os.getcwd(),'xtb_calculations')

xtb = stko.XTBEnergy(xtb_path=xtb_path,
           output_dir = output_dir+'\\'+stk.get_inchi_key(polymer),
    unlimited_memory=False,
    calculate_free_energy=False,
)

xtb_results = xtb.get_results(polymer)
# Extract properties from the energy calculator for a given
# molecule.
total_energy = xtb_results.get_total_energy()
homo_lumo_gap = xtb_results.get_homo_lumo_gap()
fermi_levels = xtb_results.get_fermi_level()
homo_lumo_orbitals = xtb_results.get_homo_lumo_orbitals()
full_dipole_moments = xtb_results.get_full_dipole_moments()

AttributeError: 'XTBExtractor' object has no attribute 'total_energy'

0 h 4 min 53.938 s 
-187.962625458816
2.193297709295


In [158]:
import uuid
import shutil
class XTBEnergy2(stko.XTBEnergy):
    def _run_xtb_2(self, xyz, out_file, init_dir, output_dir):
        """
        Runs GFN-xTB.

        Parameters
        ----------
        xyz : :class:`str`
            The name of the input structure ``.xyz`` file.

        out_file : :class:`str`
            The name of output file with xTB results.

        init_dir : :class:`str`
            The name of the current working directory.

        output_dir : :class:`str`
            The name of the directory into which files generated during
            the calculation are written.

        Returns
        -------
        None : :class:`NoneType`

        """

        # Modify the memory limit.
        if self._unlimited_memory:
            memory = 'ulimit -s unlimited ;'
        else:
            memory = ''

        if self._solvent is not None:
            solvent = f'--{self._solvent_model} {self._solvent} '
        else:
            solvent = ''

        if self._calculate_free_energy:
            hess = '--hess'
        else:
            hess = ''

        if self._calculate_ip_and_ea:
            vipea = '--vipea'
        else:
            vipea = ''

        cmd = (
            f'{memory} {self._xtb_path} '
            f'{xyz} --gfn {self._gfn_version} '
            f'{hess} {vipea} --parallel {self._num_cores} '
            f'--etemp {self._electronic_temperature} '
            f'{solvent} --chrg {self._charge} '
            f'--uhf {self._num_unpaired_electrons} -I det_control.in'
        )
        print(cmd)
        try:
            os.chdir(output_dir)
            self._write_detailed_control()
            print(output_dir)
            with open(out_file,'w') as f:
                # Note that sp.call will hold the program until
                # completion of the calculation.
                sp.call(
                    cmd,
                    stdin=sp.PIPE,
                    stdout=f,
                    stderr=sp.PIPE,
                    # Shell is required to run complex arguments.
                    shell=True
                )
        finally:
            os.chdir(init_dir)
            print(init_dir)
    def calculate2(self, mol):
        if self._output_dir is None:
            output_dir = str(uuid.uuid4().int)
        else:
            output_dir = self._output_dir
        #output_dir = os.path.abspath(output_dir)

        if os.path.exists(output_dir):
            shutil.rmtree(output_dir)
        os.mkdir(output_dir)

        init_dir = os.getcwd()
        xyz = os.path.join( output_dir,'input_structure.xyz')
        out_file = os.path.join( 'energy.output')
        mol.write(xyz)
        xyz = os.path.join( 'input_structure.xyz')

        yield self._run_xtb_2(
            xyz=xyz,
            out_file=out_file,
            init_dir=init_dir,
            output_dir=output_dir,
        )
    def get_results_2(self, mol):
            """
            Calculate the xTB properties of `mol`.

            Parameters
            ----------
            mol : :class:`.Molecule`
                The :class:`.Molecule` whose energy is to be calculated.

            Returns
            -------
            :class:`.XTBResults`
                The properties, with units, from xTB calculations.

            """

            if self._output_dir is None:
                output_dir = str(uuid.uuid4().int)
            else:
                output_dir = self._output_dir
            output_dir = os.path.abspath(output_dir)

            out_file = os.path.join(output_dir, 'energy.output')

            return stko.XTBResults(
                generator=self.calculate2(mol),
                output_file=out_file,
        )

xtb = XTBEnergy2(xtb_path=xtb_path,
           output_dir = "xtb_calculations"+'\\'+stk.get_inchi_key(polymer),
    unlimited_memory=False,
    calculate_ip_and_ea=True
)

xtb_results = xtb.get_results_2(polymer)

 "C:\Users\ma11115\OneDrive - Imperial College London\xtb-6.5.0\bin\"\xtb.exe input_structure.xyz --gfn 2  --vipea --parallel 1 --etemp 300  --chrg 0 --uhf 0 -I det_control.in
xtb_calculations\CLUOOVKTWZALSH-UHFFFAOYSA-N
c:\Users\ma11115\OneDrive - Imperial College London\github_folder\BO_polymers


In [163]:

xtb = XTBEnergy2(xtb_path=xtb_path,
           output_dir = "xtb_calculations"+'\\'+stk.get_inchi_key(polymer),
    unlimited_memory=False,
    calculate_ip_and_ea=True
)

xtb_results = xtb.get_results_2(polymer)
total_energy = xtb_results.get_total_energy()
homo_lumo_gap = xtb_results.get_homo_lumo_gap()
fermi_levels = xtb_results.get_fermi_level()
homo_lumo_orbitals = xtb_results.get_homo_lumo_orbitals()
full_dipole_moments = xtb_results.get_full_dipole_moments()
electron_affinity = xtb_results.get_electron_affinity()
ionisation_potential=xtb_results.get_ionisation_potential()

'eV'